In [3]:
topics = [
    "Дебетовые карты",
    "Кредитные карты",
    "Акции и бонусы",
    "Кредиты наличными",
    "Вклады",
    "Ипотека",
    "Автокредиты",
    "Рефинансирование кредитов",
    "Рефинансирование ипотеки",
    "Обмен валют",
    "Условия",
    "Денежные переводы",
    "Мобильное приложение",
    "РКО",
    "Эквайринг",
    "Обслуживание",
    "Дистанционное обслуживание",
    "Техподдержка и чат",
    "Другие услуги"
]


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
import json

# Список всех возможных тем
topics = [
    "Дебетовые карты",
    "Кредитные карты",
    "Акции и бонусы",
    "Кредиты наличными",
    "Вклады",
    "Ипотека",
    "Автокредиты",
    "Рефинансирование кредитов",
    "Рефинансирование ипотеки",
    "Обмен валют",
    "Условия",
    "Денежные переводы",
    "Мобильное приложение",
    "РКО",
    "Эквайринг",
    "Обслуживание",
    "Дистанционное обслуживание",
    "Техподдержка и чат",
    "Другие услуги"
]


In [5]:
class ReviewDataset(Dataset):
    def __init__(self, data, topics, device):
        self.topics = topics
        self.embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")
        self.device = device
        self.data = [item for item in data if len(item["topics_found"]) > 0]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["original_text"]
        labels = [1 if t in item["topics_found"] else 0 for t in self.topics]
        emb = self.embedder.encode(text)  # numpy
        emb = torch.tensor(emb, dtype=torch.float).to(self.device)
        return emb, torch.tensor(labels, dtype=torch.float).to(self.device)


In [6]:

# ---- Classifier ----
class Classifier(nn.Module):
    def __init__(self, embed_dim, num_labels):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_labels)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)  # BCEWithLogitsLoss


In [7]:
def train(model, dataloader, epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        total_loss = 0
        for emb, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(emb)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader):.4f}")


In [8]:
# 1. Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2. Загружаем JSON
files = ["/content/data.json", "/content/data2.json"]

# 2. Загружаем и объединяем все данные
all_data = []
for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        all_data.extend(data)  # добавляем все элементы

print(f"Всего отзывов для обучения: {len(all_data)}")
# 3. Датасет + DataLoader
dataset = ReviewDataset(all_data, topics, device)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


Using device: cuda
Всего отзывов для обучения: 3225


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
model = Classifier(embed_dim=512, num_labels=len(topics)).to(device)


In [38]:

# 4. Модель

# 5. Обучение
train(model, dataloader, epochs=50)


Using device: cuda
Всего отзывов для обучения: 3225
Epoch 1, Loss: 0.3249
Epoch 2, Loss: 0.2575
Epoch 3, Loss: 0.2322
Epoch 4, Loss: 0.2174
Epoch 5, Loss: 0.2074
Epoch 6, Loss: 0.1994
Epoch 7, Loss: 0.1930
Epoch 8, Loss: 0.1875
Epoch 9, Loss: 0.1826
Epoch 10, Loss: 0.1780
Epoch 11, Loss: 0.1740
Epoch 12, Loss: 0.1706
Epoch 13, Loss: 0.1672
Epoch 14, Loss: 0.1643
Epoch 15, Loss: 0.1616
Epoch 16, Loss: 0.1586
Epoch 17, Loss: 0.1563
Epoch 18, Loss: 0.1541
Epoch 19, Loss: 0.1518
Epoch 20, Loss: 0.1495
Epoch 21, Loss: 0.1472
Epoch 22, Loss: 0.1455
Epoch 23, Loss: 0.1434
Epoch 24, Loss: 0.1411
Epoch 25, Loss: 0.1395
Epoch 26, Loss: 0.1378
Epoch 27, Loss: 0.1355
Epoch 28, Loss: 0.1338
Epoch 29, Loss: 0.1320
Epoch 30, Loss: 0.1302
Epoch 31, Loss: 0.1287
Epoch 32, Loss: 0.1269
Epoch 33, Loss: 0.1248
Epoch 34, Loss: 0.1232
Epoch 35, Loss: 0.1215
Epoch 36, Loss: 0.1198
Epoch 37, Loss: 0.1183
Epoch 38, Loss: 0.1165
Epoch 39, Loss: 0.1147
Epoch 40, Loss: 0.1129
Epoch 41, Loss: 0.1116
Epoch 42, Loss

In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import json

# ---- Classifier с Dropout ----
class Classifier(nn.Module):
    def __init__(self, embed_dim, num_labels):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)  # Dropout 30%
        self.fc2 = nn.Linear(128, num_labels)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)  # BCEWithLogitsLoss

# ---- Функция тренировки с валидацией и ранней остановкой ----
def train(model, train_loader, val_loader, epochs=50, patience=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)  # weight_decay для регуляризации
    criterion = nn.BCEWithLogitsLoss()

    best_val_loss = float('inf')
    counter = 0

    for epoch in range(epochs):
        # --- Тренировка ---
        model.train()
        total_loss = 0
        for emb, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(emb)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)

        # --- Валидация ---
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for emb, labels in val_loader:
                outputs = model(emb)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

        # --- Ранняя остановка ---
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            counter = 0
            torch.save(model.state_dict(), "best_model.pt")  # сохраняем лучшую модель
        else:
            counter += 1
            if counter >= patience:
                print("Ранняя остановка!")
                break

# ---- Устройство ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [47]:
# 1. Создаем объект модели с теми же параметрами
model = Classifier(embed_dim=512, num_labels=len(topics)).to(device)

# 2. Загружаем веса
model.load_state_dict(torch.load("model3.pth", map_location=device))

# 3. Переводим в режим оценки (если инференс)
model.eval()

Classifier(
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=19, bias=True)
)

In [48]:
def predict(model, text, topics, threshold=0.5):
    embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")
    emb = embedder.encode([text])
    emb = torch.tensor(emb, dtype=torch.float).to(device)
    with torch.no_grad():
        logits = model(emb)
        probs = torch.sigmoid(logits).cpu().numpy()[0]
    found = [t for t, p in zip(topics, probs) if p > threshold]
    return found


In [49]:
sample_text = "Рекомендовал другу дебетовую карту, а вознаграждение так и не получил."
print(predict(model, sample_text, topics))

['Дебетовые карты', 'Акции и бонусы']


In [18]:
!pip install razdel


In [29]:
from razdel import sentenize

from collections import defaultdict

def analyze_review(model, review_id, text, topics, threshold=0.5):
    # Разбиваем на предложения
    sentences = [s.text for s in sentenize(text)]

    # Словарь для фрагментов
    topic_fragments = defaultdict(list)

    # Для каждого предложения определяем темы
    for sent in sentences:
        found = predict(model, sent, topics, threshold)
        for topic in found:
            topic_fragments[topic].append(sent)

    # Список всех найденных тем
    topics_found = list(topic_fragments.keys())

    # Преобразуем defaultdict в обычный dict с нужной структурой
    topic_fragments_dict = {}
    for topic, frags in topic_fragments.items():
        topic_fragments_dict[topic] = {"fragments": frags}

    # Собираем JSON
    result = {
        "review_id": review_id,
        "original_text": text,
        "sentences": sentences,
        "topics_found": topics_found,
        "topic_fragments": topic_fragments_dict
    }

    return result


In [51]:
sample_review = "Мобилка ужасное. Постоянно вылетает, авторизация срабатывает через раз, а переводы иногда вообще зависают. Пользоваться невозможно, нервы трепет."
review_json = analyze_review(model, review_id=1, text=sample_review, topics=topics)
print(json.dumps(review_json, ensure_ascii=False, indent=2))

{
  "review_id": 1,
  "original_text": "Мобилка ужасное. Постоянно вылетает, авторизация срабатывает через раз, а переводы иногда вообще зависают. Пользоваться невозможно, нервы трепет.",
  "sentences": [
    "Мобилка ужасное.",
    "Постоянно вылетает, авторизация срабатывает через раз, а переводы иногда вообще зависают.",
    "Пользоваться невозможно, нервы трепет."
  ],
  "topics_found": [
    "Денежные переводы",
    "Мобильное приложение"
  ],
  "topic_fragments": {
    "Денежные переводы": {
      "fragments": [
        "Мобилка ужасное.",
        "Постоянно вылетает, авторизация срабатывает через раз, а переводы иногда вообще зависают.",
        "Пользоваться невозможно, нервы трепет."
      ]
    },
    "Мобильное приложение": {
      "fragments": [
        "Пользоваться невозможно, нервы трепет."
      ]
    }
  }
}


In [44]:
# Сохраняем только веса модели
torch.save(model.state_dict(), "/content/model3.pth")
print("Модель сохранена в model.pth")


Модель сохранена в model.pth
